In [2]:
import soundfile as sf
from __future__ import  division
import numpy as np
import matplotlib.pyplot as plt
import librosa
%matplotlib inline
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU



def wave_to_spectum(x, n_fft):
    S = librosa.stft(x, n_fft) 
    A = np.log1p(np.abs(S))  
    return A




    

In [ ]:
dir = []
rootdir = r"C:\Users\Yuchen Jin\Documents\190\project\src\musdb18\train"
for subdir, dirs, files in os.walk(rootdir):
    for name in dirs:
        dir.append(rootdir+ '\\'+ name + '\mixture.wav' )

data = []

In [ ]:
data = np.zeros((0,1025,100))
for i in range(30):
    path = dir[i]
    sample, sample_rate = librosa.load(path)
    n_fft = 2048
    imdata = wave_to_spectum(sample,n_fft)
    imdata = np.pad(imdata, [(0, 0), (0, 100-np.shape(imdata)[1]%100)], mode='constant')
    data = np.concatenate((data, np.reshape(imdata,(int(np.shape(imdata)[1]/100),1025,100))), axis=0)

In [ ]:
dir = []
rootdir = r"C:\Users\Yuchen Jin\Documents\190\project\src\musdb18\train"
for subdir, dirs, files in os.walk(rootdir):
    for name in dirs:
        dir.append(rootdir+ '\\'+ name + '\\vocals.wav' )

In [ ]:
y_in = np.zeros((0,1025,1))
ind = 0
for i in range(1):
    path = dir[i]
    sample, sample_rate = librosa.load(path)
    n_fft = 2048
    imdata = wave_to_spectum(sample,n_fft)
    imdata = np.pad(imdata, [(0, 0), (0, 100-np.shape(imdata)[1]%100)], mode='constant')
    plt.pcolormesh(imdata.T[0:1000].T)
    for j in range(0,int(np.shape(imdata)[1]/100)):
        y_in = np.concatenate((y_in, (np.reshape(imdata.T[j*100+50],(1,1025,1)))), axis=0)


In [ ]:
def create_network():
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same', input_shape=(1025, 100, 1)))
    model.add(LeakyReLU())
    model.add(Conv2D(16, (3,3), padding='same'))
    model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.3))
    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(LeakyReLU())
    model.add(Conv2D(16, (3,3), padding='same'))
    model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    model.add(Dense(1025, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model


model = create_network()



In [ ]:
import gc
gc.collect()

In [ ]:
def train_network(data, y_in, model):
    """ Train a Neural Network to seperate vocal """
    
    network_input = np.reshape(data,(np.shape(data)[0],1025,100,1))
    network_output = np.reshape(y_in,(np.shape(y_in)[0],1025))
    network_output = np.where(network_output > 0.01, 1, 0)



 
    checkpoint = keras.callbacks.ModelCheckpoint(
        "weights2-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    
    callbacks_list = [checkpoint]

    print("training...")
    model.fit(network_input, network_output, batch_size=20, epochs=300, callbacks=callbacks_list)
train_network(data,y_in,model)

In [3]:

model = keras.models.load_model('src\models\ADAM_BINARYCROSSENTROPY0.0410.hdf5')



In [ ]:


path = r"C:\Users\Yuchen Jin\Documents\190\project\src\musdb18\test\Girls Under Glass - We Feel Alright\mixture.wav"
sample, sample_rate = librosa.load(path)
n_fft = 2048
indata = wave_to_spectum(sample,n_fft)


indata = np.pad(indata, [(0, 0), (0, 100-np.shape(indata)[1]%100)], mode='constant')



In [ ]:
the_stft = librosa.stft(sample,n_fft)
the_stft = np.pad(the_stft, [(0, 0), (0, 100-np.shape(the_stft)[1]%100)], mode='constant')


In [ ]:
result = np.zeros((0,1025,1))
for i in range(0,np.shape(indata)[1]-100):
    pred = model.predict(np.reshape((indata.T[i:i+100]).T,(1,1025,100,1)))
    
    result = np.concatenate((result,np.reshape(pred,(1,1025,1))), axis = 0)

In [ ]:

result2 = np.concatenate((np.zeros((49,1025,1)),result), axis = 0)
result2 = np.concatenate((result2,np.zeros((51,1025,1))), axis = 0)
result2 = np.reshape(result2,(np.shape(result2)[0],1025)).T
result2 = np.multiply(np.round(result2), the_stft)
final_result = librosa.istft(result2, win_length=n_fft)






In [ ]:

indata1 = wave_to_spectum(final_result,n_fft)
plt.pcolormesh(indata1.T[0:1000].T)

In [ ]:
plt.pcolormesh(indata.T[0:1000].T)

In [ ]:
sf.write("text.wav", final_result,sample_rate)
plt.plot(sample)
plt.plot(final_result)

In [ ]:
import museval

def calculate_sdr(ref, est, scaling=False):
    s = museval.evaluate(ref[None,:,None], est[None,:,None], win = len(ref), hop = len(ref))
    return s[0][0]


print(calculate_sdr(sample, final_result))

In [ ]:
model.summary()